<a href="https://colab.research.google.com/github/KacperGrodecki/image_proccessing/blob/main/kuba_img_text_reading_model_apply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get install poppler-utils 
!apt-get install tesseract-ocr-pol
!apt-get install libleptonica-dev 
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-dev
!apt-get install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (120 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Reading package lists... Done
Building depende

In [4]:
pip install pytesseract 

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=c2d9e08b3e586740deefcfc9e086d775e5fc11b58139a9d0b9476240ae64be78
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract


In [5]:
pip install autocorrect

     |████████████████████████████████| 622 kB 4.2 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.5.0-py3-none-any.whl size=621851 sha256=abe0899f434ae08979aeee8ed6d53be7b53272f37628425a0ee7a4258384b72e
  Stored in directory: /root/.cache/pip/wheels/3d/8e/bd/f6fd900a056a031bf710a00bca338d86f43b83f0c25ab5242f
Successfully built autocorrect


In [6]:
import pytesseract
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
import pandas as pd
import cv2
from PIL import Image,ImageTk
from pytesseract import image_to_string
import numpy as np
import re
import matplotlib.pyplot as plt      
from skimage import io, color, morphology
import pickle
from os import listdir
from os.path import isfile, join
import gc
import sys
import pickle

In [1]:
#statistics
def average_word_length(text):
    if isinstance(text, str):
        text=re.sub(r'W+', '',text)
        if len(text.strip())>0:
            total_length = len(text)-text.count(' ')
            num_words = len(text.split())
            return total_length/num_words
        else:
            return 0
    elif text==np.nan:
        return 0
    else:
        average_word_length(str(text))
        
def std(text):
    if isinstance(text, str):
        text=re.sub(r'W+', '',text)
        if len(text.strip())>0:
            total_length = len(text)-text.count(' ')
            num_words = len(text.split())
            avg=total_length/num_words
            words=text.split()
            out=0
            for word in words:
                out+=(len(word)-avg)**2 
            return (out/num_words)**0.5
        else:
            return 0
    elif text==np.nan:
        return 0
    else:
        std(str(text))
        
def longest(text):
    if isinstance(text, str):
        if len(text.strip())>0:
            words=text.split()
            out=0
            for word in words:
                if len(word)>out:
                    out=len(word)
            return out
        else:
            return 0
    elif text==np.nan:
        return 0
    else:
        std(str(text))

def char_type_counter(text):
    #print(text)
    if isinstance(text, str):
        dig,alpha=0,0
        for char in text.strip():
            if char.isdigit():
                dig+=1
            elif char.isalpha():
                alpha+=1
        if dig+alpha==0:
            return 0
        else:
            return alpha/(dig+alpha)
    else:
        return 0

def unique_chars_set(s):
    if type(s) ==str and len(s)>0:
      return  (len(s)-len(set(s)))/len(s)
    else:
      return 0

In [26]:
class Run():

    def __init__(self):
        #print('start')
        self.countter = 0
        self.countter1 = 0
        self.labels = []
        self.xywh = []
        self.xywh1 = []
        self.path = 'drive/MyDrive/kuba/zdjecia/'
        self.files = [f for f in listdir(self.path) if isfile(join(self.path, f))]
        self.files.sort()
        self.contours = 0

    def setNumber(self,i):
        self.file_nr = i

    def openFile(self):
        self.file = self.path + self.files[self.file_nr]
        return self.file

    def findContour(self,ImageProcessing):
        fig_prepared = ImageProcessing.figPrepare(self.file)
        self.contours, self.xywh = ImageProcessing.find_contours(fig_prepared)
        return self.contours

    def readText(self, cropped_board):
        crop = Image.fromarray(cropped_board)
        try:
            let_crop = image_to_string(cropped_board, lang='pol', config='--psm 7 --oem 3')
        except:
            let_crop = ''
        try:
            div_crop = image_to_string(cropped_board, lang='pol', config='--psm 6 --oem 3')
        except:
            let_crop = ''
        if re.search('[a-zA-Z]', let_crop) or re.search('[a-zA-Z]', div_crop):
            dig_crop = image_to_string(cropped_board, lang='pol', config='--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789')  # r'--oem 3 --psm 3 outputbase digits'

        let_crop=str(let_crop)[:-2]
        div_crop=str(div_crop)[:-2]
        if self.modelCheck(let_crop)==0:
            let_crop=''
        if self.modelCheck(div_crop)==0:
            div_crop=''
        
        return let_crop, div_crop
    def modelCheck(self,x):
      averages=average_word_length(x)
      stds=std(x)
      long1=longest(x)
      char_type_count=char_type_counter(x)
      unique_chars=unique_chars_set(x)
      errors=int('ERROR' in x)
      x=np.array([averages,stds,long1,char_type_count,unique_chars,errors])
      filename='drive/MyDrive/kuba/model.joblib'
      with open(filename, 'rb') as file:  
       model = pickle.load(file)
      return model.predict(np.transpose(x.reshape(-1,1)))[0]

In [8]:
class ImageProcessing:

    def figPrepare(self, file):
        rgb_image = cv2.imread(file)

        gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
        lower_blue=np.array([0])
        upper_blue=np.array([120])
        mask=cv2.inRange(gray_image,lower_blue,upper_blue)

        res=cv2.bitwise_and(gray_image,gray_image,mask=mask)
        res[mask==0]=[255]
        board=int(res.shape[1]*0.01)
        res[:,:board]=255
        res[:,-board:]=255


        return res#gray_image_copy

    def countrurs(self, gray, ythresh):
        thresh = 255 - gray
        # use morphology erode to blur horizontally
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (151, 3))
        morph = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel)

        # use morphology open to remove thin lines from dotted lines
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 17))
        morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
        cntrs = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]

        # find the topmost box

        for c in cntrs:
            box = cv2.boundingRect(c)
            x, y, w, h = box
            if y < ythresh:
                topbox = box
                ythresh = y
        
        return cntrs[::-1], topbox

    # loading contours
    def image_correction(self,gray_image):
        unique, counts = np.unique(gray_image, return_counts=True)
        hist_max=np.argmax(counts[120:180])+120
        hist_max_all=np.argmax(counts)
        
        kernelSizes = (11, 11)
        gray_image_blurred = cv2.blur(gray_image,kernelSizes)
        gray_image_copy=gray_image.copy()
        gray_image_copy[(gray_image<hist_max+20)&(gray_image>hist_max-20)]=hist_max_all+10

        return gray_image_copy

    def find_contours(self, file):
        image = file
  
        result=self.image_correction(image)####################test-podmieniona linijka wyzej#########
        ythresh = 1000
        cntrs, topbox = self.countrurs(image, ythresh)
        i = 0
        images = []
        xywh = []
        for c in cntrs:
            box = cv2.boundingRect(c)
            if box != topbox:
                i = i + 1
                x, y, w, h = box

                xywh.append([x, y, w, h])

                cv2.rectangle(result, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cropped = image[y:y + h, x:x + w]

                # mask,mask1 = self.mask_from_cropped(cropped)
                board = 50
                cropped_board = cv2.copyMakeBorder(cropped, board, board, board, board, cv2.BORDER_CONSTANT,
                                                   value=[255, 255, 255])

                images.append(cropped_board)
        return images, xywh


In [28]:
imageProcessing=ImageProcessing()
run=Run()
text_df=pd.DataFrame(np.array([' ',' ']))
text=[]
for i in range(0,600):
    print('-------------------------',i,'--------------------------')
    
    #ImageProcessing
    run.setNumber(i)
    try:
      file=run.openFile()
    except:
      break
    text.append(file)
    print(file)
    if "concat" in file:
      continue
    contours=run.findContour(imageProcessing)
    #print(len(contours))
    j=0
    for contour in contours:
        line,div=run.readText(contour)
        if line!='':
           text.append(line)
        if div!=line and div!='':
          text.append(div)
        j+=1
    #print(np.array(text).flatten())
    #text_df.append(pd.DataFrame(np.array(text).flatten()))

------------------------- 0 --------------------------
drive/MyDrive/kuba/zdjecia/fig1.jpg
------------------------- 1 --------------------------
drive/MyDrive/kuba/zdjecia/fig10.jpg
------------------------- 2 --------------------------
drive/MyDrive/kuba/zdjecia/fig100.jpg
------------------------- 3 --------------------------
drive/MyDrive/kuba/zdjecia/fig101.jpg
------------------------- 4 --------------------------
drive/MyDrive/kuba/zdjecia/fig102.jpg
------------------------- 5 --------------------------
drive/MyDrive/kuba/zdjecia/fig103.jpg
------------------------- 6 --------------------------
drive/MyDrive/kuba/zdjecia/fig104.jpg
------------------------- 7 --------------------------
drive/MyDrive/kuba/zdjecia/fig106.jpg
------------------------- 8 --------------------------
drive/MyDrive/kuba/zdjecia/fig107.jpg
------------------------- 9 --------------------------
drive/MyDrive/kuba/zdjecia/fig109.jpg
------------------------- 10 --------------------------
drive/MyDrive/kub

UnboundLocalError: ignored

In [30]:
print(len(text))
text_pd=pd.DataFrame(np.array(text))

3113


In [31]:
text_pd.to_csv('drive/MyDrive/kuba/odczyt_0_85.csv')